In [227]:
import pandas as pd
import numpy as np
import cv2
from sklearn.cluster import KMeans
from pandas import Series, DataFrame, Panel
import matplotlib.pyplot as plt
import matplotlib.figure as fig
import matplotlib.image as mpimg
from matplotlib import collections as mc 
import os
import argparse
import sys
import subprocess
import math

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

##################################### Tunable Variables ###########################################
days_to_process = 7
pd.options.display.max_rows = 10
###################################################################################################

#####################################string variables##############################################
dayArray = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17'
,'18','19','20','21','22','23','24','25','26','27','28','29','30','31']
monthArray = ['01','02','03','04','05','06','07','08','09','10','11','12',]
# Tunable Variables
# currentYear = 2013 # input 1
# pointerMonth = n n1	   # input 2
# pointerDay = 0   # input 3

currentYear = 2013
currentMonth = monthArray[0]
currentDay = dayArray[0]
pointerMonth = 0
pointerDay = 0
daysToChoose = 3
#####################################################################################################

###############################string concatenation & Load Data########################
counter = 0
frames = pd.DataFrame()
os.chdir('E:\Documents\MMU Studies\Python Scripts\Track LOST dataset')
currentIndex = 1 # This variable is used to reindex every dataframe
print(os.getcwd())
while counter < daysToChoose:
    stringDate = '001_' + str(currentYear) + str(currentMonth) + str(currentDay) + '.txt' 
    string_to_be_parsed = "pd.read_table('" + stringDate + "',delimiter=' ', header=None, names=col_names)"
    try:
        exec("%s%d = %s" % ("day", counter, string_to_be_parsed))
        tempString_Date = eval("%s%d" % ("day", counter))   
        tempString_Date, currentIndex = trackID_reindex(tempString_Date, currentIndex)
        frames = frames.append(tempString_Date)
        print("Yes!")
    except:
        counter -= 1
        print("Nope!")
    counter += 1
    currentYear, currentMonth, currentDay, pointerMonth, pointerDay = calendarFunction(currentYear, currentMonth, currentDay, pointerMonth, pointerDay)
#######################################################################################

col_names = ['TrackID', 'FrameNo', 'X', 'Y']
# concatDay = pd.concat(frames, ignore_index=True) #this shud be removed cuz frames no longer is np array, it's pd df now.
concatDay = frames.sort_values(by='TrackID', ascending=True)
concatDay = concatDay.reset_index(drop=True)
os.chdir('E:\Documents\MMU Studies\Python Scripts')

# TraClusFileExporter(concatDay, "LOST_365_Days.tra")
# dataTraining()
# ##################################Clustering##########################################

clusteredTracks = ReadTraclusExport('LOST4Output.txt')
test_subject = np.array([[200,300],[100,150]])

nj_parameter = nj_training_parameter( concatDay, clusteredTracks)
calculateSimilarity(clusteredTracks, test_subject, nj_parameter)
# average_grad_X, average_grad_Y = calculateVector(clusteredTracks)

# #For All Clusters
# cluster = TrajectoryID_Extraction(clusteredTracks)
# colors_for_lines = ColorAssignment(clusteredTracks)
# cluster_line = LinesConstruct(clusteredTracks)
# Visualizer(cluster_line, colors_for_lines)

#For Separate cluster
# cluster = TrajectoryID_Extraction(clusteredTracks)
# colors_for_lines = ColorAssignment(cluster)
# cluster_line = LinesConstruct(cluster)
# Visualizer(cluster_line, colors_for_lines)

# ######################################################################################


E:\Documents\MMU Studies\Python Scripts\Track LOST dataset
Yes!
Yes!
Yes!


C:\Users\Ice\Anaconda3\lib\site-packages\ipykernel\__main__.py:66: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


ZeroDivisionError: division by zero

In [226]:
def calculateSimilarity(representative_trajectory, new_trajectory, parameter_nj):
    
    # splitting tracks based on id
    representative_trajectory, pivot_list = trajectory_pivoting_based_on_id(representative_trajectory)
    
    representative_trajectory = pd.DataFrame.as_matrix(representative_trajectory)

    tracks_counter = 0
    probability_array = []
    
#         print((representative_trajectory[ tempNP_splitter[tracks_counter]:tempNP_splitter[tracks_counter+1] ]))
#         print(tracks)
#     for each in parameter_nj:
    while tracks_counter < len(pivot_list) - 1:
        probability_array += [math.exp( (-parameter_nj[tracks_counter]) * chamfer_distance(new_trajectory,representative_trajectory[ pivot_list[tracks_counter]:pivot_list[tracks_counter+1] ]))]
        tracks_counter += 1
#     print(parameter_nj)
    print(probability_array)
    return 1

In [223]:
def nj_training_parameter(training_track, representative_track):
    
    # calculate each of the training track chamfer distance
    parameter_nj = []
    training_track, pivot_list = trajectory_pivoting_based_on_id(training_track)
    
    representative_track, pivot_list2 = trajectory_pivoting_based_on_id(representative_track)
    
    # convert to numpy
    training_track = pd.DataFrame.as_matrix(training_track)
    representative_track = pd.DataFrame.as_matrix(representative_track)
    
    tracks_counter = 0
    representative_counter = 0
    
    
    while representative_counter < len(pivot_list2) - 1:
        bottom_param = 0
        while tracks_counter < len(pivot_list) - 1:
            bottom_param += chamfer_distance(training_track[pivot_list[tracks_counter]:pivot_list[tracks_counter + 1]],
                                             representative_track[ pivot_list2[representative_counter]:pivot_list2[representative_counter + 1] ])
            tracks_counter += 1
        if bottom_param >0:    
            combined_param = len(training_track) / bottom_param
        parameter_nj += [combined_param]
        representative_counter += 1
    print(bottom_param)
    
    return parameter_nj

In [224]:
def trajectory_pivoting_based_on_id(trajectory_dataframe):
    tempNP_splitter = []
    tempNP_splitter += [0]
    pivot = 0
    pivoted_cluster_id = trajectory_dataframe[['TrackID']].loc[pivot]
    pivoted_cluster_id = pivoted_cluster_id.TrackID
    while pivot < len(trajectory_dataframe):        
        current_cluster_id = trajectory_dataframe[['TrackID']].loc[pivot]
        current_cluster_id = current_cluster_id.TrackID
        if current_cluster_id != pivoted_cluster_id:
            tempNP_splitter += [pivot]
            pivoted_cluster_id = current_cluster_id
        pivot += 1
    tempNP_splitter += [len(trajectory_dataframe)]
    trajectory = trajectory_dataframe[['X','Y']]        
    return trajectory, tempNP_splitter

In [220]:
def chamfer_distance(track_p, track_q):
     
    first_term = ( 1 / abs(len(track_p)) )

    difference_tp_tq = []
    for points in track_p:
        temp_dist = np.subtract(track_q[:,0], points[0])
        temp_dist2 = np.subtract(track_q[:,1], points[1])
        temp_dist = np.add(temp_dist, temp_dist2)
        temp_dist = np.power(temp_dist, 2)
        temp_dist = min(temp_dist)
        difference_tp_tq += [temp_dist]
#     print(difference_tp_tq)
    second_term = sum(difference_tp_tq)    
    
    distance = first_term * second_term
    
#     print(distance)
    return distance

In [4]:
def calculateGradient(dataframeInput):
    tempnp = pd.DataFrame.as_matrix(dataframeInput[['X']])
    tempnp = tempnp.ravel()
    dataframe_gradient_X = np.gradient(tempnp)
    
    tempnp = pd.DataFrame.as_matrix(dataframeInput[['Y']])
    tempnp = tempnp.ravel()
    dataframe_gradient_Y = np.gradient(tempnp)
    
    x = dataframeInput[['X']].loc[2]
    y = dataframeInput[['Y']].loc[2]
    print(x)
    print(y)
    euclideanNorm = math.hypot(x, y)
    print(euclideanNorm)
    return dataframe_gradient_X, dataframe_gradient_Y
    
def calculateVector(dataframeInput):
    #####################Separate clusters for mean calculation################################
    tempNP_splitter = []
    pivot = 0
    pivoted_cluster_id = dataframeInput[['TrackID']].loc[pivot]
    pivoted_cluster_id = pivoted_cluster_id.TrackID
    while pivot < len(dataframeInput):        
        current_cluster_id = dataframeInput[['TrackID']].loc[pivot]
        current_cluster_id = current_cluster_id.TrackID
        if current_cluster_id != pivoted_cluster_id:
            tempNP_splitter += [pivot]
            pivoted_cluster_id = current_cluster_id
        pivot += 1
    
    counter = 0
    current_pivot = 0
    visualize_np_X = np.empty(0)
    visualize_np_Y = np.empty(0)
    average_np_X = np.empty(0)
    average_np_Y = np.empty(0)
    
    while counter <= len(tempNP_splitter):
        try:
            tempDF_X, tempDF_Y = calculateGradient(dataframeInput[current_pivot:tempNP_splitter[counter]])

        except:
            tempDF_X, tempDF_Y = calculateGradient(dataframeInput[current_pivot:(dataframeInput.iloc[-1].name + 1)])
        try:
            current_pivot = tempNP_splitter[counter]
        except:
            current_pivot = 0
        visualize_np_X = np.append(visualize_np_X, tempDF_X)
        visualize_np_Y = np.append(visualize_np_Y, tempDF_Y)
        tempDF_X = np.mean(tempDF_X)
        tempDF_Y = np.mean(tempDF_Y)
        average_np_X = np.append(average_np_X, tempDF_X)
        average_np_Y = np.append(average_np_Y, tempDF_Y)
        counter += 1

    ###########################################################################################
    
    ################## Average gradient for each cluster##############
    temp_grad_X = pd.DataFrame.mean(dataframeInput[['X']], axis = 0)
    temp_grad_Y = pd.DataFrame.mean(dataframeInput[['Y']], axis = 0)
    ##################################################################
    
    
    ######################## Plotting ###################################
    plt.gca().invert_yaxis()
    plt.quiver(dataframeInput[['X']], dataframeInput[['Y']], visualize_np_X, visualize_np_Y)
#     plt.quiver(average_np_X, average_np_Y)
    plt.show()
    #####################################################################
    return average_np_X, average_np_Y

In [5]:
def dataTraining():
    os.chdir('E:\Documents\MMU Studies\Python Scripts\TraClusAlgorithm\src')
    print("Running TraClus Algorithm...")
    subprocess.call("dir",shell=True)
#     sys.getopt(LOST_30_Days)
    os.chdir('E:\Documents\MMU Studies\Python Scripts')
    subprocess.call("echo " + os.getcwd(), shell=True)

In [6]:
def trackID_reindex(dataframe_to_be_reindexed, currentIndex):
    counter = 1
    
    ## Newly indexed Series
    trackID_Series = dataframe_to_be_reindexed[['TrackID']]
    trackID_Series.loc[0].TrackID = currentIndex
    
    ## reference Series
    ref_Series = dataframe_to_be_reindexed[['TrackID']]
    initial_ID = ref_Series.loc[0].TrackID
    
    while counter < len(dataframe_to_be_reindexed):
        if ref_Series.loc[counter].TrackID == initial_ID:
            trackID_Series.loc[counter].TrackID = currentIndex
        else:
            currentIndex += 1
            trackID_Series.loc[counter].TrackID = currentIndex
            initial_ID = ref_Series.loc[counter].TrackID
        counter += 1
    currentIndex += 1 ##for new document
    dataframe_to_be_reindexed[['TrackID']] = trackID_Series    
    return dataframe_to_be_reindexed, currentIndex

In [7]:
def calendarFunction(currentYear, currentMonth, currentDay, pointerMonth, pointerDay):
    if (pointerMonth == 3) or (pointerMonth == 5) or (pointerMonth == 8) or (pointerMonth == 10):
        if currentDay == '30':
            currentDay = dayArray[0]
            pointerMonth += 1
            currentMonth = monthArray[pointerMonth]
            pointerDay = 0
        else:
            pointerDay += 1
            currentDay = dayArray[pointerDay]
            
    elif pointerMonth == 1:
        if (currentYear%4 == 0) and (currentDay == '28'):#leapyear
            pointerDay += 1
            currentDay = dayArray[pointerDay]
        elif (currentYear%4 == 0) and (currentDay == '29'):#reset to 01 if day ends for the month
            currentDay = dayArray[0]
            pointerDay = 0
            pointerMonth += 1
            currentMonth = monthArray[pointerMonth]
        elif (currentYear%4 != 0) and (currentDay == '28'): #Non Leap year
            currentDay = dayArray[0]
            pointerDay = 0
            pointerMonth += 1
            currentMonth = monthArray[pointerMonth]
        else:
            pointerDay += 1
            currentDay = dayArray[pointerDay]
    else: #for 31 days month
        if currentDay == '30':
            pointerDay += 1
            currentDay = dayArray[pointerDay]
        elif currentDay == '31':
            currentDay = dayArray[0]
            pointerDay = 0
            pointerMonth += 1
            currentMonth = monthArray[pointerMonth]
        else:
            pointerDay += 1
            currentDay = dayArray[pointerDay]

    #if year ends?
    if (pointerMonth == 11) and (pointerDay == 30):
        currentYear += 1
        pointerMonth = 0
        pointerDay = 0
        currentMonth = monthArray[pointerMonth]
        currentDay = dayArray[pointerDay]
    return currentYear, currentMonth, currentDay, pointerMonth, pointerDay
    

In [8]:
def imageOverlaying(imageA, imageB):
    image1 = plt.imread(imageA)
    image2 = plt.imread(imageB)
    imgCombination = image1 + image2
    return imgCombination

In [9]:
# Get the output from TraClus, revamp and prepare data to visualize
# to achieve this, i need to read from text file and reconstruct it back to a dataframe.
def ReadTraclusExport(filename):
    fileOpener = open(filename, 'r')
    lines_ori = fileOpener.readlines()

    ########Choose the rows with X Y coordinates####################################
    #a loop is needed to differentiate the xy and affilitated trajectory's id
    counterLineNumber = 0 
    for lines in lines_ori:
        indexOfTraj = lines.find("trajectoryID: 0")
        if indexOfTraj == 0:
            break
        counterLineNumber += 1

    start = 2
    end = counterLineNumber 
    step = 2
    lines = lines_ori[start:end:step]
    ################################################################################

    ####################Choose the rows with ID and number of points################
    clusterID_array = []
    num_points = []
    start = 1
    end = counterLineNumber
    step = 2
    lines_ID = lines_ori[start:end:step]
    length_lines_ID = len(lines_ID)
    counter_for_linesID = 0
    
    #a global list for accessment
    global end, lines_ori
    
    while counter_for_linesID < length_lines_ID:
        temp_lines_ID = str(lines_ID[counter_for_linesID])
        temp_index = temp_lines_ID.find(" ")
        temp_index3 = temp_lines_ID.find("P")
        temp_index2 = temp_lines_ID.find("  ", 14, len(temp_lines_ID))
        clusterID_array += [temp_lines_ID[temp_index + 1:temp_index3]]
        num_points += [temp_lines_ID[temp_index2 + 2:]]
        counter_for_linesID += 1
    ##################################################################################

    #Strip the string into appropriate form
    element = str(lines)
    elementText = element[0:-1]
    elementText = str.replace(elementText, '\\' , '')
    elementText = str.replace(elementText, "n", '')
    elementText = str.replace(elementText, "   ", " ")
    elementText = str.replace(elementText, ",", "\n")
    elementText = str.replace(elementText, "\n", '')
    elementText = str.replace(elementText, "'", "" )
    elementText = str.replace(elementText, "  "," ")
    elementText = str.replace(elementText, "[", "")

    #put in 2d array form and convert to integer
    numpyString = np.fromstring(elementText, dtype=float, sep= ' ' or '\n')
    numpyInt = numpyString.astype(int)
    number_of_rows = int( len(numpyInt)/2 )
    numpyInt = numpyInt.reshape((number_of_rows,2))

    #convert numpy array to dataframe
    dataframeInt = pd.DataFrame(numpyInt)
    dataframeInt.columns = ['X', 'Y']
    dataframeInt = dataframeInt.convert_objects(convert_numeric=True)

    #Assign ID to specific tracks
    temp_array_ID = []
    total_num_points = len(num_points)
    counter_for_total_num = 0
    while counter_for_total_num < total_num_points:
        tempCounter_for_num = 0
        while tempCounter_for_num < int(num_points[counter_for_total_num]):
            temp_array_ID += [clusterID_array[counter_for_total_num]]
            tempCounter_for_num += 1
        counter_for_total_num += 1

    #finalized output
    temp_array_ID = list(map(int, temp_array_ID))
    dataframeInt['TrackID'] = temp_array_ID
    cols = dataframeInt.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    dataframeInt = dataframeInt[cols]

    return dataframeInt

<ipython-input-9-dbff7f68ef4f>:33: SyntaxWarning: name 'end' is assigned to before global declaration
  global end, lines_ori
<ipython-input-9-dbff7f68ef4f>:33: SyntaxWarning: name 'lines_ori' is assigned to before global declaration
  global end, lines_ori


In [10]:
#Decide the color for each cluster
##########stripping the trajectory id#################
def TrajectoryID_Extraction(dataframeInt):
    startTraj_ID = end + 1
    endTraj_ID = len(lines_ori)
    stepTraj_ID = 2
    traj_ID = lines_ori[startTraj_ID:endTraj_ID:stepTraj_ID]
    counter_cluster_ID = 0
    array_for_cluster_ID = []
    count_for_cluster_ID = []
    ## Form ***clusterID***

    while counter_cluster_ID < len(traj_ID):
        count_cluster_ID = 0
        number_of_string = traj_ID[counter_cluster_ID].count(' ')
        while count_cluster_ID < number_of_string:
            array_for_cluster_ID += [counter_cluster_ID]
            count_cluster_ID += 1
        counter_cluster_ID += 1
    array_for_cluster_ID = pd.DataFrame(array_for_cluster_ID)
    
    #sample: elementText = str.replace(elementText, '\\' , '')
    #sample: numpyString = np.fromstring(elementText, dtype=float, sep= ' ' or '\n')
    #sample" dataframeInt = pd.DataFrame(numpyInt)
    ## Stripping string to form ***trajectoryID***
    traj_ID = str(traj_ID)
    traj_ID = str.replace(traj_ID, '[\'', '')
    traj_ID = str.replace(traj_ID, '\']', '')
    traj_ID = str.replace(traj_ID, ', ', '\n')
    traj_ID = str.replace(traj_ID, '\\n', '')
    traj_ID = str.replace(traj_ID, '\'', '')
    traj_ID = np.fromstring(traj_ID, dtype=int, sep=' ')
    traj_ID = pd.DataFrame(traj_ID)
    traj_ID.columns = ['TrackID']
    traj_ID['ClusterID'] = array_for_cluster_ID
    ######################################################

    #check the matched value through joining
#     print(traj_ID)
    tempDF = pd.DataFrame()
    tempDF = pd.merge(concatDay, traj_ID, on='TrackID')
    return tempDF

In [11]:
def ColorAssignment(dataframe):

    try:
        dataframe[['ClusterID']]
    except:
#         dataframe = LinesConstruct(dataframe)        
        dataframe = dataframe.rename(columns={'TrackID': 'ClusterID'})
    

    ########################Assign Color##################
    tempColor = np.where(dataframe['ClusterID'] == 0, 'blue', 
                         np.where( dataframe['ClusterID'] == 1, 'yellow', 
                                  np.where(dataframe['ClusterID'] == 2, 'cyan',
                                           np.where(dataframe['ClusterID'] == 3, 'magenta','black'))))    
    counter = 1
    helper_color = tempColor[0]
    tempColor2 = []
    tempColor2 += [helper_color]
    while counter < len(tempColor):
        if helper_color != tempColor[counter]:
            tempColor2 += [tempColor[counter]]
            helper_color = tempColor[counter]
        counter += 1

    return tempColor2
    ######################################################  

In [12]:
# Construct lines for visualization in plot
def LinesConstruct(dataFrameSelected):
    concatenator = dataFrameSelected[['TrackID','X','Y']]
    concatenator2 = dataFrameSelected[['X', 'Y']]

    row_iterator = concatenator.iterrows()
    row_id, this_row = next(row_iterator)
    #slicing dataframe to respective arrays for LineCollection function
    initialID = this_row['TrackID']
    slicingArray = []
    for i, rows in row_iterator:
        if rows['TrackID'] != initialID:
            slicingArray.append(i)
            initialID = rows['TrackID']

    testChamp = np.split( concatenator2, slicingArray )
    testChampSize = len(testChamp)
    dummyTest2 = list(testChamp)
    return dummyTest2

In [13]:
# Prepare .tra extension file for TraClus clustering algorithm
def TraClusFileExporter(concatDay, filename):
    outputFile = []
    concatDay = concatDay[['TrackID', 'X', 'Y']]
    concatDayWithoutID = concatDay[['X', 'Y']]
    concatDayWithID = concatDay[['TrackID']]
    concatDayWithID = concatDayWithID.drop_duplicates('TrackID')
    concatDayWithID = concatDayWithID.values  #it's an array now


    # slicing the dataframe based on ID
    row_iterator = concatDay.iterrows()
    row_id, this_row = next(row_iterator)
    #slicing dataframe to respective arrays for LineCollection function
    initialID = this_row['TrackID']
    slicingArray = []

    ####################suspected bug################################
    for i, rows in row_iterator:
        if rows['TrackID'] != initialID:
            slicingArray.append(i)
            initialID = rows['TrackID']

    newList = np.split( concatDayWithoutID, slicingArray )

    ####################suspected bug################################

    #Now: We need to combine every single piece of data to conform with .tra
    #header: number of dimensions
    #      : number of trajectories
    number_of_dimension = 2
    number_of_trajectory = len(concatDayWithID)
    arrayForHeader = str(number_of_dimension) + '\n' + str(number_of_trajectory) + '\n'
#     print(concatDayWithID)
    #format: TrackID no_trajectory_point X1 Y1 X2 Y2... Xn Yn
    arrayForXY = []
    number_of_ID = len(concatDayWithID)
    counter_for_points = 0
    counter_for_ID = 0

    while counter_for_ID < number_of_ID:
        # reset
        counter_for_points = 0
        # Convert ID to proper string form
        tempID = concatDayWithID[counter_for_ID]
        number_of_trajectory_points = len(newList[counter_for_ID])
        stringForTempID = str(tempID)
        stringForTempID = stringForTempID[1:-1]
        # concatenation
        if len(arrayForXY) == 0:
            arrayForXY = stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
        else:
            arrayForXY += stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
        while counter_for_points < number_of_trajectory_points:  
            # Select X and Y
            tempHelper = newList[counter_for_ID]
            tempArray = tempHelper.iloc[counter_for_points]
            helperX = tempArray[0]
            helperY = tempArray[1]
            arrayForXY += str(helperX) + ' ' + str(helperY) + ' '
            counter_for_points += 1
            if counter_for_points == number_of_trajectory_points:
                arrayForXY += '\n'
        counter_for_ID += 1

    finalOutput = arrayForHeader + arrayForXY    

    output = open(filename, 'w')
    output.write(finalOutput)
    output.close()

In [14]:
def Visualizer(list_of_lines, lineColor):
    indices = [0, 1]
    counterForLineConstruction = 0

    #line preparation
    #### Visualize line with Color
    linesegment = mc.LineCollection(list_of_lines, linewidths = 2, linestyles='solid', colors=lineColor)
    #### Visualize line without Color
    # linesegment = mc.LineCollection(dataframeInt, linewidths = 2, linestyles='solid', colors='black')
    #### Visualize an arrow



    #canvas setup
    x = np.arange(641)
    ys = x[0:480, np.newaxis]

    # set plot limits
    ax = plt.axes()
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(ys.min(), ys.max())
    ax.add_collection(linesegment)
    plt.gca().invert_yaxis()
    im = plt.imread('overlayingImage.png')
    implot = plt.imshow(im)
    plt.show()